In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# %config Completer.use_jedi = False

import os
import sys
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgbm
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.nn as nn
import optuna
import optiver2023
import random
from tqdm import tqdm
random.seed(100)
env = optiver2023.make_env()
iter_test = env.iter_test()

REDUCED = True


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

print('python:', sys.version)

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


python: 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0]
/kaggle/input/optiver-trading-at-the-close/public_timeseries_testing_util.py
/kaggle/input/optiver-trading-at-the-close/train.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv
/kaggle/input/optiver-trading-at-the-close/optiver2023/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/optiver-trading-at-the-close/optiver2023/__init__.py


In [2]:
train = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')
test = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv')
sample_submission = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv')
revealed_targets = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv')

In [3]:
train.head(5)

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4


In [4]:
test.head(5)

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,time_id,row_id,currently_scored
0,0,478,0,3753451.43,-1,0.999875,11548975.43,NaN,NaN,0.999875,22940.00,1.000050,9177.60,1.0,26290,478_0_0,False
1,1,478,0,985977.11,-1,1.000245,3850033.97,NaN,NaN,0.999940,1967.90,1.000601,19692.00,1.0,26290,478_0_1,False
2,2,478,0,599128.74,1,1.000584,4359198.25,NaN,NaN,0.999918,4488.22,1.000636,34955.12,1.0,26290,478_0_2,False
3,3,478,0,2872317.54,-1,0.999802,27129551.64,NaN,NaN,0.999705,16082.04,1.000189,10314.00,1.0,26290,478_0_3,False
4,4,478,0,740059.14,-1,0.999886,8880890.78,NaN,NaN,0.999720,19012.35,1.000107,7245.60,1.0,26290,478_0_4,False


In [5]:
revealed_targets.head(5)

,stock_id,date_id,seconds_in_bucket,time_id,revealed_target,revealed_date_id,revealed_time_id
0,0.0,478,0,26290,-2.310276,477.0,26235.0
1,1.0,478,0,26290,-12.850165,477.0,26235.0
2,2.0,478,0,26290,-0.439882,477.0,26235.0
3,3.0,478,0,26290,7.259846,477.0,26235.0
4,4.0,478,0,26290,4.780292,477.0,26235.0


In [6]:
# distribution except for stock_id, date_id, seconds_in_bucket, row_id
# fig = plt.figure()
# ax = fig.add_subplot(1,1,1)
train = train.drop(['stock_id','row_id','seconds_in_bucket','row_id', 'far_price', 'near_price'], axis=1)
val = test.drop(['stock_id','row_id','seconds_in_bucket','row_id', 'far_price', 'near_price', 'currently_scored'], axis=1)
# boxplot_labels = plt_train.columns
# sns.boxplot(train['imbalance_size'])

In [7]:
# 欠損値がどのくらいあるか見てみる
# 今回のデータは幅が広そうなので中央値でnull埋め
# すべて量的データ

null_data = train.isnull()

print('data size:', len(train))
print('null sum', null_data.sum())
print('===============================')

if REDUCED: train = train[:1000] # データ量が多すぎるのでいったん1000件くらいで

data size: 5237980
null sum date_id                      0
imbalance_size             220
imbalance_buy_sell_flag      0
reference_price            220
matched_size               220
bid_price                  220
bid_size                     0
ask_price                  220
ask_size                     0
wap                        220
target                      88
time_id                      0
dtype: int64


In [8]:
fillna_train = pd.DataFrame()
col_names = train.columns
for col_name in col_names:
    fillna_train[col_name] = train[col_name].fillna(train[col_name].median()) #いったん中央値で試す

print(fillna_train.isnull().sum())
print('columns length ->', len(col_names))

date_id                    0
imbalance_size             0
imbalance_buy_sell_flag    0
reference_price            0
matched_size               0
bid_price                  0
bid_size                   0
ask_price                  0
ask_size                   0
wap                        0
target                     0
time_id                    0
dtype: int64
columns length -> 12


In [9]:
# モデル、学習用関数の定義

EPOCHS = 20
BATCH_SIZE = 32
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

class Dataset:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# NNとLightGBMは分離させる
class NeuralNetEnsembleModel(nn.Module):
    def __init__(self, params):
        super().__init__()
        self.nn_params = params

        # layer
        self.input_layer = nn.Linear(self.nn_params['input_dim'], self.nn_params['embed_dim'])
#         self.attn_layer = nn.MultiheadAttention(embed_dim=self.nn_params['embed_dim'], 
#                                                 num_heads=self.nn_params['num_heads'], 
#                                                 batch_first=True)
        self.output_layer = nn.Linear(self.nn_params['embed_dim'], self.nn_params['output_dim'])
    
    def forward(self, X):      
        y = self.input_layer(X)
#         y = self.attn_layer(y)
        y = self.output_layer(y)
        return y
                 
        # resnetを入れる？

# NNの学習
def train(model, dataloader, optimizer, loss_fn, device):
    model.train() #学習モード
    train_loss = 0

    for batch_X, batch_y in tqdm(dataloader): # ミニバッチ学習
        batch_X, batch_y = batch_X.to(device), batch_y.to(device) # gpuに送る
        optimizer.zero_grad()
        pred = model.forward(batch_X)
        pred = pred[:, 0]
        
        loss = loss_fn(pred, batch_y)
        loss.backward()
        optimizer.step() # ネットワーク更新
        train_loss += loss.item()

    return train_loss / len(dataloader)

def eval(model, dataloader, optimizer, loss_fn, device):
    model.eval() #評価モード
    eval_loss = 0
    
    for batch_X, batch_y in tqdm(dataloader):
        batch_X, batch_y = batch_X.to(device), batch_y.to(device) # gpuに送る
        pred = model.forward(batch_X)
        pred = pred[:, 0]
        
#         print(pred.shape)
#         print(batch_y.shape)

        loss = loss_fn(pred, batch_y)
        eval_loss += loss.item()
    return eval_loss / len(dataloader)
    


In [10]:
# LightGBM学習実行

train_X = fillna_train.drop('target', axis=1)
train_y = fillna_train['target']

eval_X = val
eval_y = revealed_targets['revealed_target']

# lgbm_model = lgbm.LGBMRegressor(**params['lgbm_params'])
# lgbm_model.fit(train_X, train_y)
# lgbm.plot_importance(lgbm_model, importance_type="gain")


In [11]:
# params
col_names = train_X.columns
params = {
    'lgbm_params':{
        'objective': 'regression',
        'metrics': 'mae',
        'n_estimators': 500,
        'random_state': 100,
        'learning_rate': 0.01,
        'device': 'gpu'
    },
    'nn_params':{
        'input_dim': len(col_names),
        'embed_dim': len(col_names)*2,
        'num_heads': len(col_names),
        'output_dim': 1,
        'learning_rate': 0.01
    }
}

In [12]:
# NN学習
# NNに突っ込むためにはテーブルデータをベクトルに変換

train_X = train_X.values
train_y = train_y.values
eval_X = eval_X.values
eval_y = eval_y.values[:len(eval_X)]

train_X = torch.from_numpy(train_X.astype(np.float32)).clone()
train_y = torch.from_numpy(train_y.astype(np.float32)).clone()
eval_X = torch.from_numpy(eval_X.astype(np.float32)).clone()
eval_y = torch.from_numpy(eval_y.astype(np.float32)).clone()


print('train_X', train_X.shape)
print('train_y', train_y.shape)
print('eval_X', eval_X.shape) # ([33000, 12])
print('eval_y', eval_y.shape) # ([33000])

model = NeuralNetEnsembleModel(params=params['nn_params']).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.L1Loss()

train_dataset = Dataset(train_X, train_y)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
eval_dataset = Dataset(eval_X, eval_y)
eval_dataloader = torch.utils.data.DataLoader(eval_dataset, batch_size=BATCH_SIZE, shuffle=False)

train_losses = []
eval_losses = []
for epoch in range(EPOCHS):
    
    train_loss = train(model=model, dataloader=train_dataloader, 
                            optimizer=optimizer, loss_fn=loss_fn, device=device)
    train_losses.append(train_loss)
    
    eval_loss = eval(model=model, dataloader=eval_dataloader, 
                          optimizer=optimizer, loss_fn=loss_fn, device=device)
    eval_losses.append(eval_loss)
    
    print(f'epoch:{epoch}, train loss:{train_loss}, eval_loss:{eval_loss}')

print('********************************************************************')
print(f'average loss -> train:{sum(train_losses)/len(train_losses)}, eval:{sum(eval_losses)/len(eval_losses)}')

train_X torch.Size([1000, 11])
train_y torch.Size([1000])
eval_X torch.Size([33000, 11])
eval_y torch.Size([33000])


100%|██████████| 1032/1032 [00:00<00:00, 1476.41it/s]


epoch:0, train loss:241796.54959106445, eval_loss:nan


100%|██████████| 1032/1032 [00:00<00:00, 1562.21it/s]


epoch:1, train loss:147382.02099609375, eval_loss:nan


...

epoch:17, train loss:54690.28674316406, eval_loss:nan


100%|██████████| 1032/1032 [00:00<00:00, 1575.36it/s]


epoch:18, train loss:56502.18226623535, eval_loss:nan


100%|██████████| 1032/1032 [00:00<00:00, 1533.90it/s]

epoch:19, train loss:61010.129974365234, eval_loss:nan
********************************************************************
average loss -> train:105754.75276870727, eval:nan


In [13]:
# 提出

for (test, revealed_targets, sample_prediction) in iter_test:
    sample_prediction['target'] = model.forward(eval_X.to(device))[:, 0]
    env.predict(sample_prediction)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


ValueError: Length of values (33000) does not match length of index (200)